In [1]:
import cobra
cobra_config = cobra.Configuration()

from cobra import Model, Reaction, Metabolite
import cobra.test
import os
from os.path import join

data_dir = cobra.test.data_dir

from modelseedpy.core.mseditorapi import MSEditorAPI

In [2]:
editor = MSEditorAPI()

In [3]:
## example
editor.add_ms_reaction('etc', ......)

TypeError: add_ms_reaction() missing 4 required positional arguments: 'model', 'rxn_id', 'compartments', and 'modelseed'

In [60]:
 #build from plasson string test
class MSEquation:

    def __init__(self, stoichiometry, d):
        self.equation = stoichiometry
        self.direction = d


#ASSUMPTION: user always inputs an = character, either in a =>, <=>, or <= arrow
def build_from_palsson_string(equation_string):

    def clean_ends(lst):
        for i in range(len(lst)):
            # remove whitespace from the front
            while (lst[i][0] == ' '):
                lst[i] = lst[i][1:]
            # remove whitespace from the back
            while (lst[i][-1] == ' '):
                lst[i] = lst[i][:-1]
        return lst

    def get_coef_and_group(lst, return_dict, side):
        for reactant in lst:
            coeficient = side
            if reactant[0] == '(':
                number = ''
                position = 1
                while reactant[position] != ')':
                    number += reactant[position]
                    position += 1
                coeficient = side * int(number)
                reactant = reactant[position + 1:]

            identifier = reactant[0]
            if reactant[-1] == ']':
                s = ''
                position = -2
                while reactant[position] != '[':
                    s = reactant[position] + s
                    position -= 1
                identifier = s
                reactant = reactant[:position]

            return_dict[(reactant, identifier)] = coeficient
        return return_dict

    #check direction
    reversible = False
    right = False
    left = False
    ret_str = ''
    reversible = equation_string.find('<=>') != -1
    if reversible:
        ret_str = 'reversable'
    else:  # not two ways, so check right
        right = equation_string.find('=>') != -1
        if right:
            ret_str = 'right'
        else:  # if not right, check left
            left = equation_string.find('<=') != -1
            if left: 
                ret_str = 'left'
            else:  # if not left, error
                ret_str = "error - no arrow found"


    #cpd00001 + cpd00002[e] => (2)cpd00003 + cpd00004
    # get substrings for either side of the euqation
    reactants_substring_list = equation_string[0:equation_string.find('=') - 1].split('+')
    products_substring_list = equation_string[equation_string.find('=') + 2:len(equation_string)].split('+')

    # clean up our substrings:
    clean_ends(reactants_substring_list)
    clean_ends(products_substring_list)

    variables = {}
    # add reactants to the dictionary

    get_coef_and_group(reactants_substring_list, variables, -1)
    get_coef_and_group(products_substring_list, variables, 1)

    ret_mse = MSEquation(variables, ret_str)

    return ret_mse
    
test = build_from_palsson_string('cpd00001 + cpd00002[e] <= (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

test = build_from_palsson_string('cpd00001 + cpd00002[e] <=> (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

test = build_from_palsson_string('cpd00001 + cpd00002[e] => (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

test = build_from_palsson_string('cpd00001 + cpd00002[e] = (2)cpd00003 + cpd00004')
print(test.equation)
print(test.direction)

{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
left
{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
reversable
{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
right
{('cpd00001', 'c'): -1, ('cpd00002', 'e'): -1, ('cpd00003', 'c'): 2, ('cpd00004', 'c'): 1}
error - no arrow found


In [58]:
#remove reactions test
def remove_reactions(model,rxn_id_list = []):
    model.remove_reactions(rxn_id_list)
    
model = cobra.io.read_sbml_model(join(data_dir, "iML1515.xml"))

print(len(model.reactions))
lst = ['CYTDK2','ICDHyr']
remove_reactions(model,lst)
print(len(model.reactions))

lst = ['C']
remove_reactions(model,lst)  #thows warning orrectly

2712
2710


C:\Users\jason\anaconda3\lib\site-packages\cobra\core\model.py:738: UserWarning: C not in iML1515
  warn("%s not in %s" % (reaction, self))


In [57]:
#edit biomass compound test

#assumptions: 
#compound is already present in the reaction, we are just changing the coeficient
def edit_biomass_compound(model,biomass_id,cpd_id,new_coef,rescale = 1):
    d = {}
    d[model.metabolites.get_by_id(cpd_id)] = new_coef
    model.reactions.get_by_id(biomass_id).add_metabolites(d, combine = False)
    
model = cobra.io.load_json_model("iML1515.json")
print(model.reactions[0])
print(model.reactions[0].metabolites)
print()
edit_biomass_compound(model, 'CYTDK2', 'cmp_c', 2)
print(model.reactions[0].metabolites)


#model = cobra.io.load_json_model("iML1515.json")
#print(model.reactions[0])
print(model.reactions[0].metabolites)
#print()
#edit_biomass_compound(model, 'CYTDK', 'cmp_c', 2)  #typo here, seeing what it does to a nonexistant reaction
#print(model.reactions[0].metabolites)

# ^ properly throws keyerror


#model = cobra.io.load_json_model("iML1515.json")
#print(model.reactions[0])
#print(model.reactions[0].metabolites)
#print()
#edit_biomass_compound(model, 'CYTDK2', 'cmp_', 2)  #typo here, seeing what it does to a nonexistant metabolite
#print(model.reactions[0].metabolites)

# ^ properly throws keyerror

CYTDK2: cytd_c + gtp_c --> cmp_c + gdp_c + h_c
{<Metabolite cmp_c at 0x2dcb3df2100>: 1.0, <Metabolite cytd_c at 0x2dcb3f679d0>: -1.0, <Metabolite gdp_c at 0x2dcb3e4a280>: 1.0, <Metabolite gtp_c at 0x2dcb3df21c0>: -1.0, <Metabolite h_c at 0x2dcb3f3d700>: 1.0}

{<Metabolite cmp_c at 0x2dcb3df2100>: 2, <Metabolite cytd_c at 0x2dcb3f679d0>: -1.0, <Metabolite gdp_c at 0x2dcb3e4a280>: 1.0, <Metabolite gtp_c at 0x2dcb3df21c0>: -1.0, <Metabolite h_c at 0x2dcb3f3d700>: 1.0}
{<Metabolite cmp_c at 0x2dcb3df2100>: 2, <Metabolite cytd_c at 0x2dcb3f679d0>: -1.0, <Metabolite gdp_c at 0x2dcb3e4a280>: 1.0, <Metabolite gtp_c at 0x2dcb3df21c0>: -1.0, <Metabolite h_c at 0x2dcb3f3d700>: 1.0}
CYTDK2: cytd_c + gtp_c --> cmp_c + gdp_c + h_c
{<Metabolite cmp_c at 0x2dcb3c8e250>: 1.0, <Metabolite cytd_c at 0x2dcaf4a1850>: -1.0, <Metabolite gdp_c at 0x2dcb3e14100>: 1.0, <Metabolite gtp_c at 0x2dcb3c8efa0>: -1.0, <Metabolite h_c at 0x2dcaeee80d0>: 1.0}



KeyError: 'cmp_'

In [55]:
#copy model reactions test
#how to do
#create two models
#delete function from 1
#add to the second
#check to see if reaction counts are the same

def copy_model_reactions(model,source_model,rxn_id_list = []):
    for rxnid in rxn_id_list:
        model.add_reaction(source_model.reactions.get_by_id(rxnid))
        
model = cobra.io.load_json_model("iML1515.json")
model2 = cobra.io.load_json_model("iML1515.json")
lst = ['CYTDK2']
print('model 1 reaction count',len(model.reactions))
remove_reactions(model,lst)
print('model 1 reaction count with removal',len(model.reactions))
copy_model_reactions(model, model2, lst)
print('model 1 reaction count',len(model.reactions))
print('model 2 reaction count',len(model2.reactions))

#testing on copying a id that already exists in the duplicate
model = cobra.io.load_json_model("iML1515.json")
model2 = cobra.io.load_json_model("iML1515.json")
copy_model_reactions(model, model2, lst)
print('model 1 reaction count',len(model.reactions))
print('model 2 reaction count',len(model2.reactions))

#testing on copying a id that does not exist in the origional
model = cobra.io.load_json_model("iML1515.json")
model2 = cobra.io.load_json_model("iML1515.json")
lst = ['C']
copy_model_reactions(model, model2, lst)
print('thows keyerror properly')
print('model 1 reaction count',len(model.reactions))
print('model 2 reaction count',len(model2.reactions))

model 1 reaction count 2712
model 1 reaction count with removal 2711
model 1 reaction count 2712
model 2 reaction count 2712


Ignoring reaction 'CYTDK2' since it already exists.


model 1 reaction count 2712
model 2 reaction count 2712


KeyError: 'C'

In [54]:
#edit reaction progam
#ASSUMPTIONS:
#an arrow will exist in the program, either =>, <=>, or <=
def edit_reaction(model,rxn_id,direction = None,gpr = None,genome = None):
    # Direction: =>, <=, or <=>
    if direction is not None:
        lower_bound = model.reactions.get_by_id(rxn_id).lower_bound
        upper_bound = model.reactions.get_by_id(rxn_id).upper_bound

        if lower_bound < 0 and upper_bound > 0: # rxn_id is reversible
            if direction == "=>":
                model.reactions.get_by_id(rxn_id).lower_bound = 0
            elif direction == "<=":
                model.reactions.get_by_id(rxn_id).upper_bound = 0
        elif lower_bound == 0 and upper_bound > 0: # rxn_id is forward only
            if direction == "<=":
                model.reactions.get_by_id(rxn_id).lower_bound = -1*upper_bound
                model.reactions.get_by_id(rxn_id).upper_bound = 0
            elif direction == "<=>":
                model.reactions.get_by_id(rxn_id).lower_bound = -1*upper_bound
        elif lower_bound < 0 and upper_bound == 0: # rxn_id is reverse only
            if direction == "=>":
                model.reactions.get_by_id(rxn_id).lower_bound = 0
                model.reactions.get_by_id(rxn_id).upper_bound = -1*lower_bound
            elif direction == "<=>":
                model.reactions.get_by_id(rxn_id).upper_bound = -1*lower_bound

    # Specify GPR as a string with boolean conditions (e.g. "(b0001 and b0002) or b1010").
    if gpr is not None:
        model.reactions.get_by_id(rxn_id).gene_reaction_rule = gpr
        
        
model = cobra.io.read_sbml_model(join(data_dir, "iML1515.xml"))

# Changing a reversible reaction to a forward (irreversible) reaction
# and changing the gpr
rxn = 'ICDHyr'
print('reversibility before change:',model.reactions.get_by_id(rxn).reversibility)
print('forward:',model.reactions.get_by_id(rxn).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn).reverse_variable)
print('gpr:',model.reactions.get_by_id(rxn).gene_name_reaction_rule)
edit_reaction(model, rxn, "=>", "(b0001 and b0002) or b1010")
print('reversibility after change:',model.reactions.get_by_id(rxn).reversibility)
print('forward:',model.reactions.get_by_id(rxn).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn).reverse_variable)
print('gpr:',model.reactions.get_by_id(rxn).gene_name_reaction_rule)

# Changing a forward (irreversible) reaction to a reversible reaction
print(' ')
rxn2 = 'HXPRT'
print('reversibility before change:',model.reactions.get_by_id(rxn2).reversibility)
print('forward:',model.reactions.get_by_id(rxn2).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn2).reverse_variable)
edit_reaction(model, rxn2, "<=>")
print('reversibility after change:',model.reactions.get_by_id(rxn2).reversibility)
print('forward:',model.reactions.get_by_id(rxn2).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn2).reverse_variable)

# Changing a forward (irreversible) reaction to a reverse (irreversible) reaction
print(' ')
rxn3 = 'CYTDK2'
print('reversibility before change:',model.reactions.get_by_id(rxn3).reversibility)
print('forward:',model.reactions.get_by_id(rxn3).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn3).reverse_variable)
edit_reaction(model, rxn3, "<=")
print('reversibility after change:',model.reactions.get_by_id(rxn3).reversibility)
print('forward:',model.reactions.get_by_id(rxn3).forward_variable)
print('reverse:',model.reactions.get_by_id(rxn3).reverse_variable)

reversibility before change: True
forward: 0 <= ICDHyr <= 1000.0
reverse: 0 <= ICDHyr_reverse_7f84b <= 1000.0
gpr: icd
reversibility after change: False
forward: 0 <= ICDHyr <= 1000.0
reverse: 0 <= ICDHyr_reverse_7f84b <= 0
gpr: ( and thrA) or rutC
 
reversibility before change: False
forward: 0 <= HXPRT <= 1000.0
reverse: 0 <= HXPRT_reverse_c7021 <= -0.0
reversibility after change: True
forward: 0 <= HXPRT <= 1000.0
reverse: 0 <= HXPRT_reverse_c7021 <= 1000.0
 
reversibility before change: False
forward: 0 <= CYTDK2 <= 1000.0
reverse: 0 <= CYTDK2_reverse_fd949 <= -0.0
reversibility after change: False
forward: 0 <= CYTDK2 <= 0
reverse: 0 <= CYTDK2_reverse_fd949 <= 1000.0
